In [5]:
import codecs, json, glob, os, sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import tensorflow as tf
from sklearn.metrics import confusion_matrix, jaccard_score, classification_report, f1_score
import numpy as np
import pandas as pd

from generator import label_mapping
from utils.image_processing import read_resize

from utils.plotting import plot_image_label_prediction, plot_uncertainty_heatmaps, \
    plot_uncertainty_statistics, plot_image, plot_label, plot_predictions

validation_result_dir = "/home/olle/PycharmProjects/LODE/workspace/validation_model_selection"

models = ["135590_L_20171201_586379001_17"]

configured_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15]


## load json files to evaluate

In [7]:
def read_decode_json(file_path):
    obj_text = codecs.open(file_path, 'r', encoding='utf-8').read()
    
    json_result_file = json.loads(obj_text)
    
    image = np.array(json_result_file["image"])
    prediction = np.array(json_result_file["prediction"])
    
    if len(image.shape) > 3:
        image = image[0, :, :, :]
        
    if len(prediction.shape) > 2:
        prediction = prediction[0, :, :]
    

    json_result_file["image"] = image
    json_result_file["label"] = np.array(json_result_file["label"])
    json_result_file["prediction"] = prediction
    return json_result_file

# specify model to be evaluated
model = models[0]

# set paths
model_dir = os.path.join(validation_result_dir, model)
json_result_paths = glob.glob(model_dir + f"/validation_results/*.json")

result_jsons = {}

for json_file_path in json_result_paths:
    record = json_file_path.split("/")[-1].replace(".json", "")
    result_jsons[record] = read_decode_json(json_file_path)

## plot records for publication

In [8]:
records = result_jsons.keys()

for record in records:
    record_log = result_jsons[record]
    data = [record_log["image"], record_log["label"], record_log["prediction"]]
    
    # all plotting functions
    plot_image_label_prediction(data, model_dir = model_dir, filename=record, mode="test")
    plot_image(record_log["image"], model_dir = model_dir, filename=record, mode="test")
    plot_label(record_log["label"], model_dir = model_dir, filename=record, mode="test")
    plot_predictions([record_log["prediction"]], model_dir = model_dir, filename=record, mode="test")

## create results per record result logs

In [9]:
records = result_jsons.keys()

f1_record_log = {}
f1_total_log = {}

all_labels = []
all_predictions = []

for record in records:
    record_log = result_jsons[record]
    
    labels = record_log["label"].flatten()
    predictions = record_log["prediction"].flatten()
    
    all_labels.extend(labels.tolist())
    all_predictions.extend(predictions.tolist())
    
    f1_record_log[record] = f1_score(labels, 
                         predictions, 
                         average = None, 
                         labels = configured_labels, 
                         zero_division=1)

# calculate for each log
f1_total_log = f1_score(all_labels, 
                        all_predictions, 
                        average = None, 
                        labels = configured_labels, 
                        zero_division=1)

# convert to data frames for plotting
total_f1_pd = pd.DataFrame(f1_total_log).T.replace(1.0,np.NaN)
record_f1_pd = pd.DataFrame(f1_record_log).T.replace(1.0,np.NaN)

In [10]:
total_f1_pd

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.976742,0.650451,0.974769,0.858779,0.969274,0.250573,0.808349,0.729999,0.258522,0.742337,0.843468,0.735982,0.993516,0.983169
